![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Load and preprocess the data
data = pd.read_csv('rental_info.csv')
data["rental_date"] = pd.to_datetime(data["rental_date"])
data["return_date"] = pd.to_datetime(data["return_date"])
data["rental_length_days"] = (data["return_date"] - data["rental_date"]).dt.days

# Create dummy variables for special features
data["deleted_scenes"] = 0
data.loc[data["special_features"].str.contains("Deleted Scenes", na=False), "deleted_scenes"] = 1
data["behind_the_scenes"] = 0
data.loc[data["special_features"].str.contains("Behind the Scenes", na=False), "behind_the_scenes"] = 1

# Prepare X and y
X = data.drop(["rental_date", "return_date", "rental_length_days", "special_features"], axis=1)
y = data["rental_length_days"]  

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

# Instantiate the model
gbr = GradientBoostingRegressor()

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 300],
    'max_depth': [3, 4],
    'learning_rate': [0.01, 0.1, 0.2]
}

# Instantiate GridSearchCV
grid_search = GridSearchCV(gbr, param_grid, cv=3, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Get the predictions
y_pred = best_model.predict(X_test)

# Calculate the MSE
best_mse = mean_squared_error(y_test, y_pred)

# Print the best MSE and the best parameters
print("Best Mean Squared Error:", best_mse)
print("Best Parameters:", grid_search.best_params_)

Best Mean Squared Error: 1.999412517204742
Best Parameters: {'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 300}
